In [1]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [2]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

# Generator

In [3]:
base_path = 'C:\\Users\\Deep\\Downloads\\Images\\archive\\real_vs_fake\\real-vs-fake'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + '\\train\\',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)




Found 100000 images belonging to 2 classes.


In [4]:
image_gen1 = ImageDataGenerator(rescale=1./255.)

valid_flow = image_gen1.flow_from_directory(
    base_path + '\\valid\\',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.


# Train Model DenseNet   

In [5]:
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [6]:
train_steps = 100000//32
valid_steps = 20000//32

history = model.fit(
    train_flow,
    epochs = 10,
    steps_per_epoch =train_steps,
    validation_data =valid_flow,
    validation_steps = valid_steps
)
model.save('.\\models\\DenseNet_model.h5')

Epoch 1/10
3125/3125 [==============================] - 847s 268ms/step - loss: 0.6224 - accuracy: 0.6417 - val_loss: 0.9133 - val_accuracy: 0.5576
Epoch 2/10
3125/3125 [==============================] - 770s 246ms/step - loss: 0.3993 - accuracy: 0.8188 - val_loss: 0.3226 - val_accuracy: 0.8642
Epoch 3/10
3125/3125 [==============================] - 770s 246ms/step - loss: 0.2363 - accuracy: 0.9027 - val_loss: 0.2370 - val_accuracy: 0.9033
Epoch 4/10
3125/3125 [==============================] - 771s 247ms/step - loss: 0.1489 - accuracy: 0.9416 - val_loss: 0.2056 - val_accuracy: 0.9191
Epoch 5/10
3125/3125 [==============================] - 770s 246ms/step - loss: 0.0972 - accuracy: 0.9625 - val_loss: 0.1878 - val_accuracy: 0.9222
Epoch 6/10
3125/3125 [==============================] - 770s 246ms/step - loss: 0.0685 - accuracy: 0.9744 - val_loss: 0.0792 - val_accuracy: 0.9709
Epoch 7/10
3125/3125 [==============================] - 770s 247ms/step - loss: 0.0499 - accuracy: 0.9811 - val_

# Evaluation

In [7]:
test_flow = image_gen1.flow_from_directory(
    base_path + '\\test\\',
    target_size=(224, 224),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)
y_pred = model.predict(test_flow)
y_test = test_flow.classes

Found 20000 images belonging to 2 classes.
20000/20000 [==============================] - 856s 43ms/step


In [9]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

ROC AUC Score: 0.9975173699999998
AP Score: 0.9973757115620031

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     10000
           1       0.99      0.94      0.96     10000

    accuracy                           0.97     20000
   macro avg       0.97      0.97      0.97     20000
weighted avg       0.97      0.97      0.97     20000

